In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
theFile='../data/processed/train_mq_fish_toxicity.csv'

In [3]:
datadf = pd.read_csv(theFile)
print("Data Shape")
print(datadf.shape)
print("Data Values")
datadf.head()   

Data Shape
(726, 7)
Data Values


,CIC0,SM1_Dz(Z),GATS1i,NdsCH,NdssC,MLOGP,responseLC50
0,3.459,1.470,1.483,0,2,0.491,4.486
1,3.833,0.405,1.522,0,1,3.492,2.092
2,4.717,0.223,1.288,1,1,4.804,6.384
3,3.609,0.811,1.208,0,0,3.291,4.380
4,3.103,0.134,1.016,0,0,2.193,3.210


In [4]:
datadf.columns

Index(['CIC0', 'SM1_Dz(Z)', 'GATS1i', 'NdsCH', 'NdssC', 'MLOGP',
       'responseLC50'],
      dtype='object')

In [5]:
datadf.describe().T

,count,mean,std,min,25%,50%,75%,max
CIC0,726.0,2.885248,0.754474,0.667,2.33075,2.9120,3.38825,5.926
SM1_Dz(Z),726.0,0.641672,0.427220,0.000,0.22300,0.5800,0.92300,2.071
GATS1i,726.0,1.288667,0.403235,0.396,0.94525,1.2235,1.56275,2.920
NdsCH,726.0,0.220386,0.578629,0.000,0.00000,0.0000,0.00000,4.000
NdssC,726.0,0.488981,0.864161,0.000,0.00000,0.0000,1.00000,6.000
MLOGP,726.0,2.109439,1.410530,-2.884,1.21475,2.1270,3.05025,6.203
responseLC50,726.0,4.072562,1.449854,0.053,3.17850,4.0270,4.91050,9.612


In [6]:
overview = datadf.describe()

In [7]:
overview.loc[["min", "max"]].to_json("schema_in.json")

In [44]:
numeric_feat = [ "CIC0","SM1_Dz(Z)","GATS1i","MLOGP","responseLC50" ]
category_feat = ["NdsCH","NdssC"]

In [45]:
from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()

In [46]:
dfi2 = datadf.copy(deep = True)

In [47]:
numeric_feat.remove('responseLC50')

In [48]:
dfi2[numeric_feat].head()

,CIC0,SM1_Dz(Z),GATS1i,MLOGP
0,3.459,1.470,1.483,0.491
1,3.833,0.405,1.522,3.492
2,4.717,0.223,1.288,4.804
3,3.609,0.811,1.208,3.291
4,3.103,0.134,1.016,2.193


In [49]:
dfi2[category_feat].head()

,NdsCH,NdssC
0,0,2
1,0,1
2,1,1
3,0,0
4,0,0


In [50]:
#one hot encoding
datadfv2 = pd.get_dummies(dfi2, columns = category_feat, drop_first = True)

In [51]:
datadfv2.head()

,CIC0,SM1_Dz(Z),GATS1i,MLOGP,responseLC50,NdsCH_1,NdsCH_2,NdsCH_3,NdsCH_4,NdssC_1,NdssC_2,NdssC_3,NdssC_4,NdssC_6
0,3.459,1.470,1.483,0.491,4.486,0,0,0,0,0,1,0,0,0
1,3.833,0.405,1.522,3.492,2.092,0,0,0,0,1,0,0,0,0
2,4.717,0.223,1.288,4.804,6.384,1,0,0,0,1,0,0,0,0
3,3.609,0.811,1.208,3.291,4.380,0,0,0,0,0,0,0,0,0
4,3.103,0.134,1.016,2.193,3.210,0,0,0,0,0,0,0,0,0


In [52]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [53]:
datadfv2[numeric_feat] = scaler.fit_transform(datadfv2[numeric_feat])

In [54]:
datadfv2.head(5)

,CIC0,SM1_Dz(Z),GATS1i,MLOGP,responseLC50,NdsCH_1,NdsCH_2,NdsCH_3,NdsCH_4,NdssC_1,NdssC_2,NdssC_3,NdssC_4,NdssC_6
0,0.760991,1.940217,0.482268,-1.148189,4.486,0,0,0,0,0,1,0,0,0
1,1.257042,-0.554364,0.579053,0.980847,2.092,0,0,0,0,1,0,0,0,0
2,2.429527,-0.980668,-0.001654,1.911635,6.384,1,0,0,0,1,0,0,0,0
3,0.959942,0.396622,-0.200187,0.838249,4.380,0,0,0,0,0,0,0,0,0
4,0.288813,-1.189136,-0.676665,0.059281,3.210,0,0,0,0,0,0,0,0,0


In [55]:
from sklearn.model_selection import train_test_split

In [56]:
X = datadfv2.drop(['responseLC50'], axis = 1)
y = datadfv2[['responseLC50']]

In [59]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics

In [126]:
from sklearn.model_selection import RepeatedKFold 
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [127]:
from sklearn.linear_model import ElasticNet
model = ElasticNet(alpha=1.0, l1_ratio=0.5)

In [128]:
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = np.absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()))

Mean MAE: 0.956 (0.089)


In [114]:
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor(n_estimators=25, random_state = 5,max_depth=15,max_features=1)

In [115]:
scores = cross_val_score(random_forest, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = np.absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

Mean MAE: 0.644 (0.082)


In [102]:
import xgboost as xgb
model = xgb.XGBRegressor(n_estimators=1000, max_depth=700, eta=0.1, subsample=0.7, colsample_bytree=0.8)

In [112]:
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = np.absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

Mean MAE: 0.656 (0.075)


In [124]:
from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor()

In [125]:
scores = cross_val_score(GBR, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = np.absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()))

Mean MAE: 0.658 (0.063)


In [118]:
'''
from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor()


parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
                  'subsample'    : [0.9, 0.5, 0.2, 0.1],
                  'n_estimators' : [100,500,1000, 1500],
                  'max_depth'    : [4,6,8,10]
                 }
                 

grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = cv, n_jobs=-1)
grid_GBR.fit(X, y)  


print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)
                 
'''